**Table of contents**<a id='toc0_'></a>

- [Setting up data](#toc1_1_1_1_1_)
  - [Recording coordinates with interactive figure](#toc1_1_1_1_2_)
  - [Creating extra-geometry from manual coordinates](#toc1_1_1_1_3_)
  - [Generating Eligible Trees and Road Pathway](#toc1_1_1_2_)
  - [Generating Possible Lines](#toc1_1_1_3_)
  - [Setting and rotating the slope line](#toc1_1_1_3_1_)
  - [Generating the possible lines depending on the slope](#toc1_1_1_3_2_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


##### <a id='toc1_1_1_1_1_'></a>[Setting up data](#toc0_)


In [3]:
# Package Imports
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

from shapely.geometry import LineString, Polygon, Point
from shapely.affinity import rotate, scale

from importlib import reload

# to load the interactive plotting version
#%matplotlib widget
# or regular
# %matplotlib inline
# %gui qt
%matplotlib qt

# Importing my modules
from src.main import data_loading, geometry_operations, geometry_utilities, cable_road_computation, mechanical_computations, plotting_2d

In [7]:
# tree_gdf, height_gdf = data_loading.load_processed_gdfs()
from src.tests import helper_functions

reload(helper_functions)
tree_gdf, height_gdf = helper_functions.set_up_gdfs()

In [8]:
reload(mechanical_computations)
# Brute computation of euler knicklast series

list_of_euler_max_force_lists = []
for index, row in tree_gdf.iterrows():
    temp_list = [
        mechanical_computations.euler_knicklast(bhd, height)
        for bhd, height in zip(row["diameter_series"], row["height_series"])
    ]
    list_of_euler_max_force_lists.append(temp_list)

tree_gdf["max_supported_force_series"] = list_of_euler_max_force_lists

tree_gdf["max_holding_force"] = (((tree_gdf["BHD"] * 0.1) ** 2) / 3) * 10000

##### <a id='toc1_1_1_1_2_'></a>[Recording coordinates with interactive figure](#toc0_)


In [4]:
# coords = []
# fig, ax = plt.subplots()
# ax.set_xlabel('X coordinate', fontsize=15)
# ax.set_ylabel('Y coordinate', fontsize=15)
# ax.scatter(tree_gdf["x"],tree_gdf["y"])

# cid = fig.canvas.mpl_connect('button_press_event',  lambda event: plotting.onclick(event,coords))

In [5]:
# disconnect the interactive figure so it doesnt eat up resources
# fig.canvas.mpl_disconnect(cid)

##### <a id='toc1_1_1_1_3_'></a>[Creating extra-geometry from manual coordinates](#toc0_)


In [9]:
# manually insert those coordinates
# recorded coordinates for the given areas
downhill_anchors = Polygon(
    [
        Point(xy)
        for xy in [
            (-123.88268627035482, 83.05479790035122),
            (-74.89307684787903, 138.5299549848453),
            (-21.10056454084676, 161.97821725767267),
            (-6.051468835903222, 148.2524051955298),
            (-60.16417466857257, 116.79741921978575),
            (-115.23746107815322, 64.75371515082742),
        ]
    ]
)
uphill_anchors = Polygon(
    [
        Point(xy)
        for xy in [
            (-55.04107825837903, -15.31352187833928),
            (-17.578435758838708, -17.601157222029755),
            (4.194723984483858, -1.5877098161964227),
            (30.450593086725803, -30.18315161232738),
            (-11.49475877173387, -41.04941949485714),
            (-58.5632070403871, -38.189875315244045),
        ]
    ]
)
# road = LineString([(-57.282432937838706, -17.029248386107135), (-34.86888614324194, -21.604519073488092), (-15.337081079379033, -15.31352187833928), (-0.2879853744354932, -7.878707011345227), (12.51975565104837, 6.990922722642864), (19.884206740701615, 21.86055245663097)])
# road = LineString([(-59.52378761729838, -12.453977698726185), (-41.91314370725806, -13.597795370571419), (-18.218822810112897, -10.16634235503571), (3.2341434075725886, 0.12801669157144602), (15.401497381782264, 23.576278964398824), (17.00246500996775, 26.43582314401192)])
road = LineString(
    [
        (-89.6286010742188, -24.4624996185303),
        (-69.618896484375, -20.364200592041),
        (-41.314998626709, -13.9251003265381),
        (-29.2644004821777, -8.47910022735596),
        (-20.2950000762939, -3.67030000686646),
        (-15.375, -0.148599997162819),
        (-6.49030017852783, 2.74860000610352),
        (0.943400025367737, 3.86450004577637),
        (15.1605997085571, -0.302300006151199),
        (29.3377990722656, -3.47709989547729),
    ]
)

inner_forest = Polygon(
    [
        Point(xy)
        for xy in [
            (-105.43005197808363, 62.73802056488026),
            (-10.557896233758669, 146.34023494632802),
            (13.160142702322588, 51.07259530235265),
            (8.890895693827957, 9.27148811162877),
            (-55.62217021231302, -10.17088732591722),
            (-81.71201304200238, 18.992675830401765),
        ]
    ]
)

extra_geometry_gpd = gpd.GeoDataFrame(
    pd.DataFrame(
        {
            "name": ["uphill_anchors", "road", "downhill_anchors", "inner_forest"],
            "geometry": [uphill_anchors, road, downhill_anchors, inner_forest],
        }
    )
)

# set the name as index to easily select the geometry
extra_geometry_gpd.set_index("name", inplace=True)
extra_geometry_gpd.loc["road"]

# plotting.plot_gdfs([tree_gdf,extra_geometry_gpd])

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


geometry    LINESTRING (-89.62860 -24.46250, -69.61890 -20...
Name: road, dtype: geometry

#### <a id='toc1_1_1_2_'></a>[Generating Eligible Trees and Road Pathway](#toc0_)


Getting the points along the road


In [10]:
interval = 2
road_points = geometry_operations.generate_road_points(
    extra_geometry_gpd.loc["road"].geometry, interval
)
# plotting.plot_scatter_xy(road_points)

In [80]:
%store road_points

Stored 'road_points' (list)


Getting the trees contained in the anchor/target tree polygons


In [12]:
# get the eligible anchor and target trees inside the polygon
anchor_trees_gdf = geometry_operations.filter_gdf_by_contained_elements(
    tree_gdf, extra_geometry_gpd.loc["uphill_anchors"].geometry
)
target_trees_gdf = geometry_operations.filter_gdf_by_contained_elements(
    tree_gdf, extra_geometry_gpd.loc["downhill_anchors"].geometry
)
inner_forest_gdf = geometry_operations.filter_gdf_by_contained_elements(
    tree_gdf, extra_geometry_gpd.loc["inner_forest"].geometry
)
harvesteable_trees_gdf = pd.concat([target_trees_gdf, inner_forest_gdf])

# filter the anchor and target trees for a BHD of 30cm or larger
anchor_trees_gdf = anchor_trees_gdf[anchor_trees_gdf["BHD"] >= 30]
target_trees_gdf = target_trees_gdf[target_trees_gdf["BHD"] >= 30]

# and check if the anchor trees look good
# plotting_2d.plot_scatter_xy(harvesteable_trees_gdf.geometry)

In [9]:
%store harvesteable_trees_gdf
%store anchor_trees_gdf
%store target_trees_gdf

Stored 'harvesteable_trees_gdf' (GeoDataFrame)
Stored 'anchor_trees_gdf' (GeoDataFrame)
Stored 'target_trees_gdf' (GeoDataFrame)


#### <a id='toc1_1_1_3_'></a>[Generating Possible Lines](#toc0_)


##### <a id='toc1_1_1_3_1_'></a>[Setting and rotating the slope line](#toc0_)


In [13]:
slope_degree = 29

# set a orientation line we can plan the line deviation around
slope_line = LineString([(0, 0), (0, 1)])
slope_line = rotate(slope_line, slope_degree)

# scale the line by a factor of 100 and plot it
slope_line = scale(slope_line, 100, 100)

# ax = plotting.plot_equal_axis(slope_line)
# ax.scatter(bestand_3_csv["x"], bestand_3_csv["y"])

In [10]:
%store slope_line

Stored 'slope_line' (LineString)


##### <a id='toc1_1_1_3_2_'></a>[Generating the possible lines depending on the slope](#toc0_)


In [75]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [36]:
plt.close("all")

In [16]:
reload(geometry_operations)
reload(geometry_utilities)
reload(plotting_2d)
reload(cable_road_computation)
reload(mechanical_computations)

plot_possible_lines = False

# # expression for debugging
# %lprun -T tmp0.txt -f cable_road_computation.create_left_end_segments_and_support_tree line_gdf, start_point_dict = cable_road_computation.generate_possible_lines(road_points, target_trees_gdf, anchor_trees_gdf, tree_gdf, slope_line, height_gdf, plot_possible_lines)
line_gdf, start_point_dict = cable_road_computation.generate_possible_lines(
    road_points,
    target_trees_gdf,
    anchor_trees_gdf,
    tree_gdf,
    slope_line,
    height_gdf,
    plot_possible_lines,
)
print("we have n lines: ", len(line_gdf))

# unpack to geopandas
buffer = gpd.GeoSeries(line_gdf["line_candidates"].values)
new_line_gdf = gpd.GeoDataFrame(geometry=buffer)
new_line_gdf[line_gdf.columns] = line_gdf[line_gdf.columns].values
new_line_gdf["line_length"] = new_line_gdf.geometry.length
line_gdf = new_line_gdf

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


2304  candidates initially
2278  after slope deviations
1388  after supports trees
75  after possible anchor triples
Cable road created from line:  (-29.880646898378075, -8.757599103801436) to  (-106.0132132, 89.40781657)
Tension to begin with is 85000.0
checking if tower and anchor trees hold
exerted force 85000.0
did not find anchor tree that holds - iterating
exerted force 85000.0
did not find anchor tree that holds - iterating
exerted force 85000.0
found anchor tree that holds
Tower and anchors hold: True
After the iterative process it is now 85000.0
checking if no collisions overall line
Found collisions
Need to find supports
Setting up support candidates
Cable road created from line:  (-29.880646898378075, -8.757599103801436) to  (-83.54479257, 57.81202571)
Cable road created from line:  (-83.54479257, 57.81202571) to  (-106.0132132, 89.40781657)
raising height to  6
checking if support withstands tension
Angle between lines 5.846905063186824
Angle between lines 5.077198619063296

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/

Cable road created from line:  (-77.01644229, 87.13747124) to  (-89.62130667, 107.525112)
raising height to  6
checking if support withstands tension
Angle between lines 7.2457538458586725
Angle between lines 7.3055193433964405
forces on lr support 10742.13072390429 10830.615971431023
checking if no collisions overall line
checking if no collisions overall line
iterating through height series since we have collisions
raising height to  7
checking if support withstands tension
Angle between lines 3.477586654457099
Angle between lines 3.6242872956975134
forces on lr support 5158.311787615679 5375.842072475513
checking if no collisions overall line
checking if no collisions overall line
iterating through height series since we have collisions
raising height to  8
checking if support withstands tension
Angle between lines 3.998067878416691
Angle between lines 3.036149779304793
forces on lr support 5930.049824621012 4503.691889784215
checking if no collisions overall line
checking if no col

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/

...

Cable road created from line:  (-16.94963143, 71.74829257) to  (-26.01718061, 121.7202521)
raising height to  6
checking if support withstands tension
Angle between lines 15.172746407731267
Angle between lines 13.02936433040935
forces on lr support 14522.270761841906 12480.359644155998
checking if no collisions overall line
checking if no collisions overall line
iterating through height series since we have collisions
raising height to  7
checking if support withstands tension
Angle between lines 18.12618848278578
Angle between lines 15.580052803767273
forces on lr support 17327.42160543606 14909.741967635
checking if no collisions overall line
checking if no collisions overall line
iterating through height series since we have collisions
raising height to  8
checking if support withstands tension
Angle between lines 20.241173824600963
Angle between lines 17.755658309945574
forces on lr support 19329.250161483727 16976.088741006748
checking if no collisions overall line
checking if no 

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/

raising height to  6
checking if support withstands tension
Angle between lines 27.000261088791916
Angle between lines 23.241636524950906
forces on lr support 25679.233726480354 22157.721719454777
checking if no collisions overall line
checking if no collisions overall line
Cable road created from line:  (-8.689096627, 10.01766072) to  (-8.603829565, 22.22068)
Cable road created from line:  (-8.603829565, 22.22068) to  (-26.01718061, 121.7202521)
raising height to  6
checking if support withstands tension
Angle between lines 28.257137074924373
Angle between lines 24.67077158760131
forces on lr support 26850.843168317057 23499.714435742735
checking if no collisions overall line
checking if no collisions overall line
Cable road created from line:  (-8.689096627, 10.01766072) to  (-26.4252543, 121.042114)
Cable road created from line:  (-26.4252543, 121.042114) to  (-26.01718061, 121.7202521)
candidate too close to anchor, skipping in check if no coll overall line sideways CR
didnt find s

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/src/main/classes_cable_road_computation.py:273: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  y_x_deflections = mechanical_computations.pestal_load_path(self, loaded)
/home/carlo/GitHub/

candidate too close to anchor, skipping in check if no coll overall line sideways CR
didnt find suitable candidate - select first candidate as support and iterate
Cable road created from line:  (-26.01718061, 121.7202521) to  (-26.4252543, 121.042114)
Cable road created from line:  (-26.4252543, 121.042114) to  (-29.84199559, 140.2430958)
recursion counter exceeded, returning
recursion counter exceeded, returning
found enough supports and returning successful cable road
found enough supports and returning successful cable road
found enough supports and returning successful cable road
75  after checking for height obstructions
we have n lines:  75


In [19]:
line_gdf_19_10_2023 = line_gdf.copy()
%store line_gdf_19_10_2023
# len(line_gdf)

# plotting_2d.plot_gdf_with_anchors_and_supports([target_trees_gdf,anchor_trees_gdf, inner_forest_gdf, extra_geometry_gpd], line_gdf_v1)

Stored 'line_gdf_19_10_2023' (GeoDataFrame)


In [19]:
line_gdf_75 = line_gdf.copy()
%store line_gdf_75

Stored 'line_gdf_75' (GeoDataFrame)


In [18]:
%store line_gdf_v1

Stored 'line_gdf_v1' (GeoDataFrame)
